# Import Modules

In [175]:
import numpy as np
import pandas as pd
import torch

import utils

# Load Data

In [221]:
target_vegetable = utils.VEGETABLES[11]

In [237]:
train_test = pd.read_csv("./data/mapped_train_test.csv")
train_test["date"] = pd.to_datetime(train_test["date"], format="%Y-%m-%d")
weather = pd.read_csv("./data/sorted_mapped_adjusted_weather.csv")
train_test = pd.concat([train_test, weather], axis=1)
train_test = train_test.reset_index(drop=True)

train_test["year"] = train_test.date.dt.year
years = pd.get_dummies(train_test["year"])
train_test = train_test.drop(columns="year")
train_test = pd.concat([train_test, years], axis=1)

train_test["month"] = train_test.date.dt.month
months = pd.get_dummies(train_test["month"])
train_test = train_test.drop(columns="month")
train_test = pd.concat([train_test, months], axis=1)

# train_test["weekday"] = train_test.date.dt.weekday
# weekdays = pd.get_dummies(train_test["weekday"])
# train_test = train_test.drop(columns="weekday")
# train_test = pd.concat([train_test, weekdays], axis=1)

areas = pd.get_dummies(train_test["area"])
train_test = train_test.drop(columns="area")
train_test = pd.concat([train_test, areas], axis=1)

test_size = pd.read_csv("./data/test.csv").shape[0]
train = train_test[:-test_size]
test = train_test[-test_size:]

train = utils.get_target_values(train, target_vegetable)
test = utils.get_target_values(test, target_vegetable)

changed_col = [1, 0] + [i for i in np.arange(2, train.shape[1])]
train = train[:, changed_col]
test = test[:, changed_col]

test_amount = pd.read_csv("./data/test_amount.csv")
test_amount = test_amount[target_vegetable].values
test[:, 1] = test_amount

train_loader, train, test, ss = utils.preprocess_data_submit(train, test)

# Set Free Params

In [238]:
future = test.shape[0]
num_epochs = 70
learning_rate = 0.001
weight_decay = 1e-3

# Training

In [245]:
model = utils.pipeline_rnn_submit(train_loader, train, test, future=future, num_epochs=num_epochs,
                                  lr=learning_rate, weight_decay=weight_decay)

# Predict

In [246]:
with torch.no_grad():
    model.eval()
    pred_y = model.predict(train, test, future)

pred_y = pred_y.cpu().detach().numpy()
pred_y = np.concatenate([pred_y.reshape(-1, 1), test[:, 1:]], axis=1)
pred_y[:, :7] = ss.inverse_transform(pred_y[:, :7])

print(f"std: {np.std(pred_y[:, 0].tolist())}")
pred_y[:, 0].tolist()

std: 29.508152253551952


[3702.580012103832,
 3686.97185907216,
 3638.036275604445,
 3647.1306710798717,
 3605.5945440036044,
 3616.3350478691573,
 3596.138737142758,
 3620.2344357769152,
 3617.455101909394,
 3603.9611251687884,
 3607.389393257241,
 3596.424814332728,
 3600.531893316641,
 3608.4708967497004,
 3584.739822046611,
 3590.6441596933973,
 3598.5887850813406,
 3605.479213614835,
 3616.439134348158,
 3625.472170201704]